In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
timers = []
timers_import_start = time.time() 
# Trainer (has labels)
#trainer_df = pd.read_csv('../datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
# learner_tweets_df = pd.read_csv('datasets/cleaned_OLID.tsv', sep="\t")
learner_tweets_df = pd.read_csv('datasets/cleaned_hatespeech_offensive.tsv', sep="\t")

timers_import_end = time.time()
print(timers_import_end - timers_import_start)

0.020359277725219727


In [2]:
import numpy as np
# trainer_labels = trainer_df['label'].values
# trainer_tweets = trainer_df['tweet'].values
#learner_tweets_df['labels'] = learner_tweets_df['average'].apply(lambda x: 1 if x >= 0.8 else 0) # threshold the average values
# learner_tweets_df['subtask_a'] = learner_tweets_df['subtask_a'].map({'OFF': 1, 'NOT': 0})

# learner_tweets_df['labels'] = learner_tweets_df["subtask_a"]
# learner_tweets_df.drop(["subtask_a","subtask_b","subtask_c"], axis=1, inplace=True)
# sample_size = 40000
positive_ratio = 0.5

# Select the most confident positive values
#semi_tweets_pos_df = learner_tweets_df[learner_tweets_df['average'] > 0.8].sample(n=np.floor(sample_size*positive_ratio).astype(int), random_state=1)

# Select the most confident negative values
#semi_tweets_neg_df = learner_tweets_df[learner_tweets_df['average'] < 0.3].sample(n=np.floor(sample_size*(1-positive_ratio)).astype(int), random_state=1)

#semi_tweets_df = pd.concat([semi_tweets_pos_df, semi_tweets_neg_df])

# Count the number of positive and negative samples
positive_samples = learner_tweets_df[learner_tweets_df['label'] == 1]
negative_samples = learner_tweets_df[learner_tweets_df['label'] == 0]

# Determine the minimum number of samples between the two classes
min_samples = min(len(positive_samples), len(negative_samples))

# Sample an equal number of positive and negative samples
balanced_samples = pd.concat([
    positive_samples.sample(n=min_samples, random_state=42),
    negative_samples.sample(n=min_samples, random_state=42)
])

# Update the semi_tweets_df with the balanced samples
semi_tweets_df = balanced_samples

semi_tweets_df = semi_tweets_df.sample(frac=1, random_state=42)


semi_tweets = semi_tweets_df['tweet'].values
semi_labels = semi_tweets_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(semi_tweets, semi_labels, test_size=0.2)


In [3]:
semi_tweets_df.head(10)

,id,label,tweet
20495,20941,0,real talk not being byes or nothin the whole n...
15840,16209,1,rt joebudden women all over america are overst...
16704,17089,1,rt nothinglikeit these hoes getting dressed up...
2251,2296,1,touchdown with on the clock michigan some hoes
18559,18968,0,rt amyalkon school admins take hungry kids lun...
9475,9735,0,going up to the view point and seeing star chi...
5012,5161,0,toddirwin nfl forgetabout they got until febru...
18011,18415,1,rt woridstarhlphop when a bad bitch asks you t...
13743,14079,0,overripe eggplants are not good eats uncletweet
4191,4314,1,mr_lewis_ brookebrooklyn_ ohh yaa he wants bro...


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
learner_dataset = TweetDataset(X_train, y_train)
test_dataset = TweetDataset(X_test, y_test)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
learner_loader = DataLoader(learner_dataset, batch_size=6, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=False)

In [5]:
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from transformers import BertTokenizer, BertForSequenceClassification

# Number of epochs

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Initialize the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


# Set up the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



/home/cymn/miniconda3/envs/nvidia-newest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
model.train()
num_epochs = 1
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(learner_loader)
    num_batches = len(learner_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

  0%|          | 0/1110 [00:00<?, ?it/s]/tmp/ipykernel_14489/692897503.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████| 1110/1110 [01:02<00:00, 17.71it/s]


In [7]:

prediction_list = np.array([])
model.eval()
correct = 0
total = 0
num_epochs = 1
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(test_loader)
    num_batches = len(test_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == labels).sum().item()
            # loss = criterion(logits, labels)
            total += labels.size(0)
            prediction_list = np.append(prediction_list, predictions.detach().cpu().numpy())
            # loss.backward()
            # optimizer.step()
            # optimizer.zero_grad()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy}')

  0%|          | 0/278 [00:00<?, ?it/s]/tmp/ipykernel_14489/2973375349.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████| 278/278 [00:03<00:00, 85.53it/s]

Validation Accuracy: 0.9561824729891957


In [8]:
from sklearn.metrics import classification_report

# Assuming you have the true labels in `val_labels` and the predicted labels in `prediction_list`
report = classification_report(y_test, prediction_list)

print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       839
           1       0.97      0.94      0.96       827

    accuracy                           0.96      1666
   macro avg       0.96      0.96      0.96      1666
weighted avg       0.96      0.96      0.96      1666



In [9]:
from sklearn.metrics import confusion_matrix

# Assuming you have the true labels in `val_labels` and the predicted labels in `prediction_list`
cm = confusion_matrix(y_test, prediction_list)

# Extract TP, TN, FP, FN from the confusion matrix
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]

print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")


True Positives (TP): 780
True Negatives (TN): 813
False Positives (FP): 26
False Negatives (FN): 47


In [10]:
# model.save_pretrained('models/HSO_BERT_2')